In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics.cluster import contingency_matrix
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import KMeans

In [2]:
# optimized partitional models
clara_opt = pd.read_csv('../2_models/with_clean_data/wind/results/partitional_opt/csv_labels_raw/clara.csv')
fuzzy_opt = pd.read_csv('../2_models/with_clean_data/wind/results/partitional_opt/csv_labels_raw/fuzzy_c_means.csv')
kmeans_opt = pd.read_csv('../2_models/with_clean_data/wind/results/partitional_opt/csv_labels_raw/kmeans.csv')

# partitionalal models
clara = pd.read_csv('../2_models/with_clean_data/wind/results/partitional/csv_labels_raw/clara.csv')
fuzzy = pd.read_csv('../2_models/with_clean_data/wind/results/partitional/csv_labels_raw/fuzzy_c_means.csv')
kmeans = pd.read_csv('../2_models/with_clean_data/wind/results/partitional/csv_labels_raw/kmeans.csv')

# model based
bgmm = pd.read_csv('../2_models/with_clean_data/wind/results/model_based/csv_labels_cut_raw/bgmm.csv')
dpmm = pd.read_csv('../2_models/with_clean_data/wind/results/model_based/csv_labels_cut_raw/dpmm.csv')
gaussian_mix = pd.read_csv('../2_models/with_clean_data/wind/results/model_based/csv_labels_cut_raw/gaussian_mix.csv')

# optimized hierarchical models
average_link_opt = pd.read_csv('../2_models/with_clean_data/wind/results/hierarchical_opt/csv_labels_cut_raw/average_link.csv')
birch_opt = pd.read_csv('../2_models/with_clean_data/wind/results/hierarchical_opt/csv_labels_cut_raw/birch.csv')
single_link_opt = pd.read_csv('../2_models/with_clean_data/wind/results/hierarchical_opt/csv_labels_cut_raw/single_link.csv')
centroid_link_opt = pd.read_csv('../2_models/with_clean_data/wind/results/hierarchical_opt/csv_labels_cut_raw/centroid_link.csv')
ward_link_opt = pd.read_csv('../2_models/with_clean_data/wind/results/hierarchical_opt/csv_labels_cut_raw/ward_link.csv')

# hierarchical models
average_link = pd.read_csv('../2_models/with_clean_data/wind/results/hierarchical/csv_labels_cut_raw/average_link.csv')
birch = pd.read_csv('../2_models/with_clean_data/wind/results/hierarchical/csv_labels_cut_raw/birch.csv')
single_link = pd.read_csv('../2_models/with_clean_data/wind/results/hierarchical/csv_labels_cut_raw/single_link.csv')
centroid_link = pd.read_csv('../2_models/with_clean_data/wind/results/hierarchical/csv_labels_cut_raw/centroid_link.csv')
ward_link = pd.read_csv('../2_models/with_clean_data/wind/results/hierarchical/csv_labels_cut_raw/ward_link.csv')

# density based models
dbscan = pd.read_csv('../2_models/with_clean_data/wind/results/density_based/csv_labels_cut_raw/DBSCAN.csv')
optics = pd.read_csv('../2_models/with_clean_data/wind/results/density_based/csv_labels_cut_raw/OPTICS.csv')

In [3]:
# Define a function to compute evaluation metrics
def compute_metrics(df):
    metrics = {}
    
    # Silhouette Score
    if df['cluster_label'].nunique() > 1:
        metrics['Silhouette Score'] = silhouette_score(df[['PC1', 'PC2']], df['cluster_label'])
        # Davies-Bouldin Index
        metrics['Davies-Bouldin Index'] = davies_bouldin_score(df[['PC1', 'PC2']], df['cluster_label'])
        # Calinski-Harabasz Index
        metrics['Calinski-Harabasz Index'] = calinski_harabasz_score(df[['PC1', 'PC2']], df['cluster_label'])
        # Dunn Index
        min_intercluster_distances = []
        max_intracluster_distances = []
        for label in df['cluster_label'].unique():
            cluster_points = df[df['cluster_label'] == label][['PC1', 'PC2']]
            cluster_distances = pdist(cluster_points)
            num_clusters = min(len(df['cluster_label'].unique()), len(cluster_points))
            kmeans = KMeans(n_clusters=num_clusters)
            kmeans.fit(cluster_points)
            min_intercluster_distances.append(min([np.linalg.norm(cluster_points.values - centroid) for centroid in kmeans.cluster_centers_]))
            if len(cluster_distances) > 0:  # Check if cluster_distances has any elements
                max_intracluster_distances.append(max(cluster_distances))
            else:
                max_intracluster_distances.append(0)  # Append a default value
        metrics['Dunn Index'] = min(min_intercluster_distances) / max(max_intracluster_distances)
        
        # Gap Statistic
        def compute_gap_statistic(data, k_max=5, n_refs=5):
            def compute_reference_distribution(data, k):
                return np.mean([
                    np.log(pdist(KMeans(k).fit(data).cluster_centers_).mean())
                    for _ in range(n_refs)
                ])
            
            observed_log_wk = np.log(pdist(KMeans(k_max).fit(data).cluster_centers_).mean())
            reference_log_wk = compute_reference_distribution(data, k_max)
            gap = reference_log_wk - observed_log_wk
            
            return gap
        
        metrics['Gap Statistic'] = compute_gap_statistic(df[['PC1', 'PC2']])
        
        # Xie-Beni Index
        # Compute the total scatter
        total_scatter = np.sum(pdist(df[['PC1', 'PC2']]) ** 2) / (2 * len(df))
        
        # Compute the intra-cluster scatter
        cluster_labels = df['cluster_label'].unique()
        intra_cluster_scatter = 0
        for label in cluster_labels:
            cluster_points = df[df['cluster_label'] == label][['PC1', 'PC2']]
            cluster_center = np.mean(cluster_points, axis=0)
            intra_cluster_scatter += np.sum(np.linalg.norm(cluster_points - cluster_center, axis=1) ** 2)
        intra_cluster_scatter /= len(df)
        
        # Compute the Xie-Beni Index
        metrics['Xie-Beni Index'] = intra_cluster_scatter / total_scatter
    else:
        metrics['Silhouette Score'] = 0
        metrics['Davies-Bouldin Index'] = 0
        metrics['Calinski-Harabasz Index'] = 0
        metrics['Dunn Index'] = 0
        metrics['Gap Statistic'] = 0
        metrics['Xie-Beni Index'] = 0

    return metrics


In [4]:
# Create a dictionary with dataframes and their names
dataframes = {
    'clara_opt': clara_opt, 'fuzzy_opt': fuzzy_opt, 'kmeans_opt': kmeans_opt,
    'clara': clara, 'fuzzy': fuzzy, 'kmeans': kmeans,
    'bgmm': bgmm, 'dpmm': dpmm, 'gaussian_mix': gaussian_mix,
    'average_link_opt': average_link_opt, 'birch_opt': birch_opt,
    'single_link_opt': single_link_opt, 'centroid_link_opt': centroid_link_opt,
    'ward_link_opt': ward_link_opt,
    'average_link': average_link, 'birch': birch, 'single_link': single_link,
    'centroid_link': centroid_link, 'ward_link': ward_link,
    'dbscan': dbscan, 'optics': optics
 }

In [5]:
# Iterate over each dataframe and print the name of the ones without a 'cluster_label' column
for model_name, df in dataframes.items():
    if 'cluster_label' not in df.columns:
        print(f"The dataframe '{model_name}' does not have a column named 'cluster_label'.")
        print(df.columns)

The dataframe 'fuzzy_opt' does not have a column named 'cluster_label'.
Index(['PC1', 'PC2', 'cluster', 'asset_id'], dtype='object')
The dataframe 'kmeans_opt' does not have a column named 'cluster_label'.
Index(['PC1', 'PC2', 'cluster', 'asset_id'], dtype='object')
The dataframe 'clara' does not have a column named 'cluster_label'.
Index(['PC1', 'PC2', 'cluster', 'asset_id'], dtype='object')
The dataframe 'fuzzy' does not have a column named 'cluster_label'.
Index(['PC1', 'PC2', 'cluster', 'asset_id'], dtype='object')
The dataframe 'kmeans' does not have a column named 'cluster_label'.
Index(['PC1', 'PC2', 'cluster', 'asset_id'], dtype='object')


In [6]:
# Loop through each dataframe and rename the 'cluster' column to 'cluster_label'
for model_name, df in dataframes.items():
    if 'cluster' in df.columns:
        df.rename(columns={'cluster': 'cluster_label'}, inplace=True)

In [7]:
# Initialize an empty list to store the results
results = []

# Compute and store the evaluation metrics for each dataframe
for model_name, df in dataframes.items():
    print(f'Evaluating {model_name}')
    metrics = compute_metrics(df)
    results.append({'Model': model_name, **metrics})

# Create a DataFrame from the results list
results_df = pd.DataFrame(results)

Evaluating clara_opt


Evaluating fuzzy_opt
Evaluating kmeans_opt
Evaluating clara
Evaluating fuzzy
Evaluating kmeans
Evaluating bgmm
Evaluating dpmm
Evaluating gaussian_mix
Evaluating average_link_opt
Evaluating birch_opt
Evaluating single_link_opt
Evaluating centroid_link_opt
Evaluating ward_link_opt
Evaluating average_link
Evaluating birch
Evaluating single_link
Evaluating centroid_link
Evaluating ward_link
Evaluating dbscan
Evaluating optics


In [8]:
# # Optionally, handle NaN values if any
# results_df = results_df.fillna(np.nan)

# Display the results
results_df

,Model,Silhouette Score,Davies-Bouldin Index,Calinski-Harabasz Index,Dunn Index,Gap Statistic,Xie-Beni Index
0,clara_opt,0.294570,0.947067,26265.623800,9.935756,-0.001603,0.000016
1,fuzzy_opt,0.322685,1.275782,21230.563071,24.509980,-0.003890,0.000030
2,kmeans_opt,0.382494,0.873168,34122.210623,19.508157,-0.000496,0.000018
3,clara,0.294570,0.947067,26265.623800,9.917686,0.001164,0.000016
4,fuzzy,0.356425,0.904400,32408.015468,15.300693,-0.000457,0.000014
5,kmeans,0.353255,0.893050,32743.503392,14.372407,-0.001142,0.000014
6,bgmm,0.096478,1.495102,11495.889168,2.679023,0.000598,0.000016
7,dpmm,0.067576,1.328895,13950.770286,0.000000,-0.000062,0.000005
8,gaussian_mix,0.355617,0.817321,34746.580918,6.254236,0.001859,0.000007
9,average_link_opt,0.678148,0.227147,16.004912,0.000000,0.000888,0.000045


In [9]:
results_df.to_csv('results/clean_wind.csv', index=False)